# UAVSAR Interferogram Processing

## Setup

In [35]:
import glob
import os
from datetime import datetime
import itertools

In [4]:
cd /u/sar-r2/fwilliam/Calif/UAVSAR/Stacks/Haywrd_05502_01-S1

/u/sar-r2/fwilliam/Calif/UAVSAR/Stacks/Haywrd_05502_01-S1


## Download UAVSAR SLC files
Unfortunately, downloading directly to the jpl server results in in a `connection denied` error, so the files must first be downloaded to the local machine, then transferred to the JPL server via the secure copy program (`scp`)

In [14]:
# Create the batch script for downloading the data
with open('Haywrd_0550201.sh','r') as f:
    lines = f.readlines()

files = [x.split(' ')[1][:-1] for x in lines]
names = [x.split('/')[-1] for x in files]

batch_lines = ['@ECHO OFF\n'] + [f'CALL CURL {x} -o {y}\n' for x,y in zip(files,names)]

with open('..\\Haywrd_0550201.bat', 'w+') as f:
    f.writelines(batch_lines)

In [27]:
# Create the batch script for transferring the data
complete = glob.glob('..\Hayw*')
remote_path = 'fwilliam@137.78.249.109:/u/sar-r2/fwilliam/Calif/UAVSAR/Stacks/Haywrd_05502_01-S1'
transfer_lines = ['@ECHO OFF\n'] + [f'CALL scp {os.path.basename(x)} {remote_path}\n' for x in complete]
with open('..\\transfer.bat', 'w+') as f:
    f.writelines(transfer_lines)

## Perform ISCE Processing

In [5]:
!mkdir downloadHH

In [7]:
!mv Haywar* downloadHH/

In [8]:
!mv downloadHH/*.dop .

In [21]:
# !prepareUAVSAR_coregStack.py -h
!prepareUAVSAR_coregStack.py -i downloadHH -o SLC -d Haywar_05502_01_BC.dop -s 1

This is the Open Source version of ISCE.
Some of the workflows depend on a separate licensed package.
To obtain the licensed package, please make a request for ISCE
through the website: https://download.jpl.nasa.gov/ops/request/index.cfm.
Alternatively, if you are a member, or can become a member of WinSAR
you may be able to obtain access to a version of the licensed sofware at
https://winsar.unavco.org/software/isce
20101110
downloadHH/Haywar_05502_10080_003_101110_L090HH_01_BC.ann
unpackFrame_UAVSAR.py -i downloadHH/Haywar_05502_10080_003_101110_L090HH_01_BC.ann -d Haywar_05502_01_BC.dop -s 1 -o /u/sar-r2/fwilliam/Calif/UAVSAR/Stacks/Haywrd_05502_01-S1/SLC/20101110
This is the Open Source version of ISCE.
Some of the workflows depend on a separate licensed package.
To obtain the licensed package, please make a request for ISCE
through the website: https://download.jpl.nasa.gov/ops/request/index.cfm.
Alternatively, if you are a member, or can become a member of WinSAR
you may be able 

Some directory setup needs to be done before we can run `stackStripMap.py`:

In [22]:
ls

dop.txt  downloadHH/  Haywar_05502_01_BC.dop  isce.log  JPL_internship/  SLC/


In [12]:
!mkdir merged

In [13]:
cd merged

/u/sar-r2/fwilliam/Calif/UAVSAR/Stacks/Haywrd_05502_01-S1/merged


In [15]:
!ln -s ../SLC

In [16]:
cd ..

/u/sar-r2/fwilliam/Calif/UAVSAR/Stacks/Haywrd_05502_01-S1


In [23]:
!cp -r /u/sar-r0/fielding/Calif/SF_Bay/topo/USGS/1_3asec/ ./dem

In [34]:
!stackStripMap.py -s SLC/ -d ./dem/BayArea_USGS_13.dem.wgs84 -a 8 -r 2 -W interferogram --nofocus -S uavsar_stack -t 600 --filter_strength 0.2

This is the Open Source version of ISCE.
Some of the workflows depend on a separate licensed package.
To obtain the licensed package, please make a request for ISCE
through the website: https://download.jpl.nasa.gov/ops/request/index.cfm.
Alternatively, if you are a member, or can become a member of WinSAR
you may be able to obtain access to a version of the licensed sofware at
https://winsar.unavco.org/software/isce
Sensor is not hard-coded (ers, envisat, alos), will keep default alks
['/u/sar-r2/fwilliam/Calif/UAVSAR/Stacks/Haywrd_05502_01-S1/SLC/20100115', '/u/sar-r2/fwilliam/Calif/UAVSAR/Stacks/Haywrd_05502_01-S1/SLC/20200910', '/u/sar-r2/fwilliam/Calif/UAVSAR/Stacks/Haywrd_05502_01-S1/SLC/20171020', '/u/sar-r2/fwilliam/Calif/UAVSAR/Stacks/Haywrd_05502_01-S1/SLC/20170403', '/u/sar-r2/fwilliam/Calif/UAVSAR/Stacks/Haywrd_05502_01-S1/SLC/20090220', '/u/sar-r2/fwilliam/Calif/UAVSAR/Stacks/Haywrd_05502_01-S1/SLC/20201007', '/u/sar-r2/fwilliam/Calif/UAVSAR/Stacks/Haywrd_05502_01-S1/SLC/2

It looks like 600 is not a large enough time delta to create a connected network, lets determine what the minimum time delta needed is:

In [45]:
d = ['20090220', '20091119', '20100115', '20100301', '20100423', '20101110', '20110712', '20111103', '20120413', '20121102', '20140528', '20141112', '20151008', '20160324', '20170403', '20171020', '20180801', '20200910', '20201007']
dates = [datetime.strptime(x,'%Y%m%d') for x in d]
dates.sort()
difference = [x-y for x,y in zip(dates,[dates[0]]+dates[0:-1])]
max(difference)

datetime.timedelta(771)

In [46]:
!stackStripMap.py -s SLC/ -d ./dem/BayArea_USGS_13.dem.wgs84 -a 8 -r 2 -W interferogram --nofocus -S uavsar_stack -t 800 --filter_strength 0.2

This is the Open Source version of ISCE.
Some of the workflows depend on a separate licensed package.
To obtain the licensed package, please make a request for ISCE
through the website: https://download.jpl.nasa.gov/ops/request/index.cfm.
Alternatively, if you are a member, or can become a member of WinSAR
you may be able to obtain access to a version of the licensed sofware at
https://winsar.unavco.org/software/isce
Sensor is not hard-coded (ers, envisat, alos), will keep default alks
['/u/sar-r2/fwilliam/Calif/UAVSAR/Stacks/Haywrd_05502_01-S1/SLC/20100115', '/u/sar-r2/fwilliam/Calif/UAVSAR/Stacks/Haywrd_05502_01-S1/SLC/20200910', '/u/sar-r2/fwilliam/Calif/UAVSAR/Stacks/Haywrd_05502_01-S1/SLC/20171020', '/u/sar-r2/fwilliam/Calif/UAVSAR/Stacks/Haywrd_05502_01-S1/SLC/20170403', '/u/sar-r2/fwilliam/Calif/UAVSAR/Stacks/Haywrd_05502_01-S1/SLC/20090220', '/u/sar-r2/fwilliam/Calif/UAVSAR/Stacks/Haywrd_05502_01-S1/SLC/20201007', '/u/sar-r2/fwilliam/Calif/UAVSAR/Stacks/Haywrd_05502_01-S1/SLC/2

In [49]:
ls run_files\

run_01_reference                  run_05_invertMisreg
run_01_reference.job              run_05_invertMisreg.job
run_02_focus_split                run_06_fineResamp
run_02_focus_split.job            run_06_fineResamp.job
run_03_geo2rdr_coarseResamp       run_07_grid_baseline
run_03_geo2rdr_coarseResamp.job   run_07_grid_baseline.job
run_04_refineSecondaryTiming      run_08_igram
run_04_refineSecondaryTiming.job  run_08_igram.job


In [47]:
!run.py -h

usage: run.py [-h] -i INPUT [-p PROCESSORS]

Preparing input rdf and processing workflow for phase unwrapping

optional arguments:
  -h, --help            show this help message and exit
  -i INPUT, --input INPUT
                        input run file which contains multiple commands to be
                        run. Each line is assumed to be a command that can be
                        run independent of other commands
  -p PROCESSORS, --number_of_processors PROCESSORS
                        number of processors
